In [4]:
import pandas as pd
from newspaper import Article
import re
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords
from collections import Counter,OrderedDict
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [5]:
f = open("news articles/articles.txt","r")
LOA = [line.rstrip() for line in f]
df = pd.DataFrame(columns = ['Title','Text'])
for url in LOA: 
    try:
        news_article = Article(url, language="en")     
        news_article.download() 
        news_article.parse() 
        news_article.nlp() 
        df2={'Title':news_article.title,'Text':news_article.text}
        df=df.append(df2,ignore_index=True)
    except Exception:
        print(url)
        
        
df.to_csv("news articles/parsed_news_articles.csv",encoding="utf-8")

https://www.economist.com/leaders/2019/10/24/narendra-modi-is-damaging-indias-economy-as-well-as-its-democracy
https://www.economist.com/graphic-detail/2020/01/24/indias-democracy-in-dan


In [6]:
def preprocessor(df):
    lemmatizer = WordNetLemmatizer() 
    stop_words = stopwords.words('english')
    text=df['Text']
    titles=df['Title']
    cleanedArticles=[]
    spcl= '[-,;@_!#$%^&*()<>?/\|}{~:''.+]'
    for i in text:
        i = i.lower()
        i = re.sub(r'[^\x00-\x7F]+',' ', i)
        i = re.sub('^a-z0-9',' ',i)
        i = re.sub(spcl,' ',i)
        i = re.sub('[0-9]',' ',i)
        i = re.sub('\n\n',' ',i)
        i = re.sub('\s\s',' ',i)
        v = i.split(' ')
        v = [word for word in v if word not in stop_words]
        v = [lemmatizer.lemmatize(word) for word in v]
        temp = ' '.join(v)
        cleanedArticles.append(temp)
    return cleanedArticles

In [7]:
cleanedArticles=preprocessor(df)

In [8]:
cleanedArticles

['since employment unemployment figure always estimated considering principal subsidiary status employment bd employment estimate based principal status incomplete misleading  representational image since employment unemployment figure always estimated considering principal subsidiary status employment bd employment estimate based principal status incomplete misleading  representational image report paper ie november  various employment estimate     based plf periodic labour force survey data received considerable attention received several query policy maker regarding difference employment estimate study laveesh bhandari amaresh dubey bd  himanshu quoted report article clarify reason difference highlight major finding study first estimate different bd based usual principal status ups employment study based usual principal subsidiary status upss employment since himanshu estimate also based upss figure much different except     plf figure himanshu estimate employment         almost neg

In [12]:
#Testing
f = open("news articles/test_articles.txt","r")
LOA = [line.rstrip() for line in f]
test_df = pd.DataFrame(columns = ['Title','Text'])
for url in LOA: 
    news_article = Article(url, language="en")     
    news_article.download() 
    news_article.parse() 
    news_article.nlp() 
    df2={'Title':news_article.title,'Text':news_article.text}
    test_df=test_df.append(df2,ignore_index=True)

test_df.to_csv("news articles/parsed_test_articles.csv",encoding="utf-8")
cleanedTestArticles=preprocessor(test_df)

In [13]:
def vectorize_sim_search(query, data):
    #Inserting query in list position 0
    cos_sim=[]
    data = query + cleanedArticles
    tfidf_vectorizer = TfidfVectorizer()
    dataMatrixAll = []
  
    for i in range(len(query)):
        query_i = query[i]
        print("MAIN ARTICLE:\n",query_i[0:len(query_i)//20],"....","\n\n\n")
        print("RELATED ARTICLES:\n")
        data = [query[i]]+cleanedArticles
        dataMatrix = {}
        tfidf_matrix = tfidf_vectorizer.fit_transform(data)
        vals = cosine_similarity(tfidf_matrix[i], tfidf_matrix)[0]
        for j in range(len(data)):
            dataMatrix[data[j]]=vals[j]
        key_list = list(dataMatrix.keys()) 
        val_list = list(dataMatrix.values()) 
        sorted_vals = sorted(vals,reverse=True)[1:5]
        for i in sorted_vals:
            temp=key_list[val_list.index(i)]
            print(temp[0:len(temp)//20],"....","\tsimilarity score:",i,"\n")
        print("-------------------------------------------------------------------------------------------")

In [15]:
vectorize_sim_search(cleanedTestArticles,cleanedArticles)

MAIN ARTICLE:
 nsso's annual household survey     first demonetisation   highlight unemployment rate   .... 



RELATED ARTICLES:

nsso's annual household survey     first demonetisation   highlight unemployment rate   .... 	similarity score: 1.0000000000000002 

report national sample survey office periodic  .... 	similarity score: 0.48348946300801293 

niti aayog thursday debunked claim jobless growth national sample survey office nsso periodic labour .... 	similarity score: 0.4473581094144747 

nearly  lakh people india lost job      report said highlighting beginning decline coincid .... 	similarity score: 0.3949835135609901 

-------------------------------------------------------------------------------------------
